## Week 3 assignment Part 3/3 
by **Morteza Ghorbani Kari**

In this notebook, I sometimes used comments for describing the code.

### Importing Libraries

In [18]:
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#### Reading data

In [17]:
neighborhoods = pd.read_csv("Toronto_Neighborhoods.csv")

# Inspecting imported data
neighborhoods.describe(include="all")

,PostCode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103.000000,103.000000
unique,103,11,102,NaN,NaN
top,M4V,North York,Queen's Park,NaN,NaN
freq,1,24,2,NaN,NaN
mean,NaN,NaN,NaN,43.704608,-79.397153
std,NaN,NaN,NaN,0.052463,0.097146
min,NaN,NaN,NaN,43.602414,-79.615819
25%,NaN,NaN,NaN,43.660567,-79.464763
50%,NaN,NaN,NaN,43.696948,-79.388790
75%,NaN,NaN,NaN,43.745320,-79.340923


### Calculating Lat. & Lng. for the map, then creating the map

In [3]:
# Finding center of the map by calculating mean of neighborhoods' lat lng

latitude = neighborhoods.Latitude.mean()
longitude = neighborhoods.Longitude.mean()

In [4]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood, code in zip(neighborhoods['Latitude'], neighborhoods['Longitude'],
                                                 neighborhoods['Borough'], neighborhoods['Neighborhood'],
                                                 neighborhoods['PostCode']):
    label = f'{code}, {borough} - {neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ff0000',
        fill=True,
        fill_color='#ffff00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

### Setting up Foursquare credentials

In [5]:
CLIENT_ID = 'AECEZ05OKNUM4DDQZPMRZ3FNDFXEWNFMHKAGCOUAYSCR4CJ1'
CLIENT_SECRET = 'M3EOELPIXED54OA0QFAX0MTJNH3KLW5V1ZBYVOUE0OW4DEYD'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AECEZ05OKNUM4DDQZPMRZ3FNDFXEWNFMHKAGCOUAYSCR4CJ1
CLIENT_SECRET:M3EOELPIXED54OA0QFAX0MTJNH3KLW5V1ZBYVOUE0OW4DEYD


### Function for retrieiving data from Foursquare
I passed PostCode to the funtion so I am able to query or join the results with PostCode too.

In [7]:
LIMIT = 100

def getNearbyVenues(code, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, name, lat, lng in zip(code, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(f'----contained {len(results)} venues----')

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostCode',
                             'Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    return(nearby_venues)

### Retreiving venue data from Foursquare

In [8]:
all_venues = getNearbyVenues(code=neighborhoods['PostCode'],
                             names=neighborhoods['Neighborhood'],
                             latitudes=neighborhoods['Latitude'],
                             longitudes=neighborhoods['Longitude']
                            )

Malvern, Rouge
----contained 2 venues----
Highland Creek, Port Union, Rouge Hill
----contained 1 venues----
Guildwood, Morningside, West Hill
----contained 8 venues----
Woburn
----contained 4 venues----
Cedarbrae
----contained 8 venues----
Scarborough Village
----contained 2 venues----
East Birchmount Park, Ionview, Kennedy Park
----contained 4 venues----
Clairlea, Golden Mile, Oakridge
----contained 9 venues----
Cliffcrest, Cliffside, Scarborough Village West
----contained 2 venues----
Birch Cliff, Cliffside West
----contained 4 venues----
Dorset Park, Scarborough Town Centre, Wexford Heights
----contained 7 venues----
Maryvale, Wexford
----contained 6 venues----
Agincourt
----contained 4 venues----
Clarks Corners, Sullivan, Tam O'Shanter
----contained 10 venues----
Agincourt North, L'Amoreaux East, Milliken, Steeles East
----contained 2 venues----
L'Amoreaux West
----contained 13 venues----
Upper Rouge
----contained 0 venues----
Hillcrest Village
----contained 5 venues----
Fairview, 

### Neighboroods without any venues

In [9]:
# Coordinates which Foursquare didn't find any venues for

neighborhoods[[i not in all_venues['PostCode'].unique()
               for i in neighborhoods['PostCode'].unique()]]

,PostCode,Borough,Neighborhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242


### Saving data to CSV file
This action has no purpose other enabling me to analyze data in other times without being online.

In [10]:
all_venues.to_csv("all_venues.csv", index=False)

### Data Shape

In [11]:
all_venues.shape

(2234, 8)

### Here we see the number of data points for each neighborhood

In [19]:
all_venues.groupby(['PostCode', 'Neighborhood']).count()

,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostCode,Neighborhood,,,,,,
M1B,"Malvern, Rouge",2,2,2,2,2,2
M1C,"Highland Creek, Port Union, Rouge Hill",1,1,1,1,1,1
M1E,"Guildwood, Morningside, West Hill",8,8,8,8,8,8
M1G,Woburn,4,4,4,4,4,4
M1H,Cedarbrae,8,8,8,8,8,8
M1J,Scarborough Village,2,2,2,2,2,2
M1K,"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4
M1L,"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
M1M,"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2


### Number of unique venue Categories

In [13]:
len(all_venues["Venue Category"].unique())

274

### Creating a One-Hot-Encoded dataframe based on existing venue categories in neighborhoods

In [20]:
# Getting the dummies
toronto_onehot = pd.get_dummies(all_venues['Venue Category'], prefix="", prefix_sep="")

# Adding *PostCode* & *Neighborhood* Columns for later use as group index
toronto_onehot[['PostCode', 'Neighborhood']] = all_venues[['PostCode', 'Neighborhood']]

# Correcting column order
fixed_columns = ['PostCode', 'Neighborhood'] + list(toronto_onehot.columns.drop(['PostCode', 'Neighborhood']))
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostCode,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,"Malvern, Rouge",0,0,0,0,0,0,0,0,

### Shape of oneHotEncoded data

In [21]:
toronto_onehot.shape

(2234, 275)

### Grouping data by Neighborhood and calculating mean values for each category

In [22]:
toronto_grouped = toronto_onehot.groupby(['PostCode', 'Neighborhood']).mean().reset_index()
toronto_grouped.head()

,PostCode,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,"Malvern, Rouge",0.0,0.0,0.0,0.0,

### Looking at 5 top categories in each neighborhood

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped.drop('PostCode', 1)[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Malvern, Rouge----
                             venue  freq
0                       Print Shop   0.5
1             Fast Food Restaurant   0.5
2                Accessories Store   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Highland Creek, Port Union, Rouge Hill----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Guildwood, Morningside, West Hill----
                 venue  freq
0          Pizza Place  0.12
1       Medical Center  0.12
2         Intersection  0.12
3  Rental Car Location  0.12
4    Electronics Store  0.12


----Woburn----
                venue  freq
0         Coffee Shop  0.50
1   Indian Restaurant  0.25
2   Korean Restaurant  0.25
3   Accessories Store  0.00
4  Miscellaneous Shop  0.00


----Cedarbrae----
             

            venue  freq
0     Coffee Shop  0.16
1          Bakery  0.06
2             Pub  0.06
3            Park  0.06
4  Breakfast Spot  0.04


----Garden District, Ryerson----
                  venue  freq
0           Coffee Shop  0.08
1        Clothing Store  0.07
2        Cosmetics Shop  0.03
3  Fast Food Restaurant  0.03
4                  Café  0.03


----St. James Town----
            venue  freq
0            Café  0.06
1     Coffee Shop  0.06
2      Restaurant  0.05
3           Hotel  0.05
4  Clothing Store  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1              Bakery  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4        Cocktail Bar  0.03


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.05
2      Ice Cream Shop  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.04


----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07


### **return_most_common_venues** function
Returns **n** top categories in a neighborhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### creating a dataframe of most common categories in neighborhoods
**neighborhoods_venues_sorted** is the resulting dataframe

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostCode', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['PostCode', 'Neighborhood']] = toronto_grouped[['PostCode', 'Neighborhood']]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostCode,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M1C,"Highland Creek, Port Union, Rouge Hill",Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
2,M1E,"Guildwood, Morningside, West Hill",Spa,Pizza Place,Electronics Store,Breakfast Spot,Rental Car Location,Intersection,Mexican Restaurant,Medical Center,Yoga Studio,Doner Restaurant
3,M1G,Woburn,Coffee Shop,Indian Restaurant,Korean Restaurant,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
4,M1H,Cedarbrae,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Gas Station,Donut Shop,Dog Run


### Clustering Neighborhoods into 5 groups (with KMeans algorithm)

In [27]:
kclusters = 5

# Dropping string columns
toronto_grouped_clustering = toronto_grouped.drop(['PostCode', 'Neighborhood'], 1)

# Creating the model
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 3, 2, 4, 2, 0, 0, 1, 2, 0, 0, 2], dtype=int32)

### Creating "toronto_merged" dataframe with Neighborhood Name, cluster number & most common venue categories 

In [28]:
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

toronto_merged = neighborhoods

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index(['PostCode', 'Neighborhood']), on=['PostCode', 'Neighborhood'])

# Removing rows with * Cluster Label = NaN *
toronto_merged = toronto_merged[toronto_merged["Cluster Labels"].notnull()]

### Visualizing Clusters on the map

In [29]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

latitude = toronto_merged.Latitude.mean()
longitude = toronto_merged.Longitude.mean()

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' - Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

### This function prints out the most common venue categories in a cluster

In [35]:
def most_common(cluster):
    a = []
    for index, row in cluster.iterrows():
        for i in range(10):
            a.append(row[i].split('\n'))

    cluster_pop = pd.Series(a).value_counts()
    print(f"Most Common Venues in this cluster are:\n\n{cluster_pop[:10]}")

### Cluster 1

In [36]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,
                   toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]

most_common(cluster1)

Most Common Venues in this cluster are:

[Yoga Studio]             8
[Dog Run]                 8
[Discount Store]          7
[Donut Shop]              7
[Doner Restaurant]        7
[Pizza Place]             7
[Diner]                   5
[Bus Line]                5
[Fast Food Restaurant]    5
[Drugstore]               5
dtype: int64


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",0.0,Fast Food Restaurant,Print Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
7,"Clairlea, Golden Mile, Oakridge",0.0,Bus Line,Bakery,Fast Food Restaurant,Intersection,Bus Station,Soccer Field,Park,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
13,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,Pizza Place,Pharmacy,Noodle House,Bank,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Dumpling Restaurant
32,Downsview Central,0.0,Food Truck,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Diner
35,"Parkview Hill, Woodbine Gardens",0.0,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Café,Bus Line,Bank
44,Lawrence Park,0.0,Photography Studio,Park,Swim School,Lawyer,Bus Line,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant
72,Glencairn,0.0,Park,Pizza Place,Asian Restaurant,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run
81,"Runnymede, The Junction North",0.0,Pizza Place,Grocery Store,Bus Line,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
96,Humber Summit,0.0,Pizza Place,Empanada Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
97,"Emery, Humberlea",0.0,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


### Cluster 2

In [37]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,
                   toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]

most_common(cluster2)

Most Common Venues in this cluster are:

[Dog Run]                        8
[Doner Restaurant]               8
[Donut Shop]                     8
[Park]                           8
[Discount Store]                 8
[Yoga Studio]                    7
[Drugstore]                      6
[Dumpling Restaurant]            5
[Eastern European Restaurant]    4
[Dim Sum Restaurant]             3
dtype: int64


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",1.0,Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
25,Parkwoods,1.0,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
30,"CFB Toronto, Downsview East",1.0,Airport,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
40,East Toronto,1.0,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
50,Rosedale,1.0,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,Caledonia-Fairbanks,1.0,Park,Women's Store,Fast Food Restaurant,Market,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
90,"Montgomery Road, Old Mill North, The Kingsway",1.0,River,Park,Pool,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
98,Weston,1.0,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


### Cluster 3

In [38]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,
                   toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]

most_common(cluster3)

Most Common Venues in this cluster are:

[Coffee Shop]         45
[Café]                30
[Doner Restaurant]    23
[Restaurant]          23
[Dog Run]             22
[Bakery]              22
[Donut Shop]          21
[Discount Store]      20
[Drugstore]           19
[Sandwich Place]      19
dtype: int64


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Port Union, Rouge Hill",2.0,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
2,"Guildwood, Morningside, West Hill",2.0,Spa,Pizza Place,Electronics Store,Breakfast Spot,Rental Car Location,Intersection,Mexican Restaurant,Medical Center,Yoga Studio,Doner Restaurant
3,Woburn,2.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
4,Cedarbrae,2.0,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Gas Station,Donut Shop,Dog Run
5,Scarborough Village,2.0,Grocery Store,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,"East Birchmount Park, Ionview, Kennedy Park",2.0,Department Store,Discount Store,Playground,Coffee Shop,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore
8,"Cliffcrest, Cliffside, Scarborough Village West",2.0,American Restaurant,Motel,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
9,"Birch Cliff, Cliffside West",2.0,Café,General Entertainment,Skating Rink,College Stadium,Concert Hall,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
10,"Dorset Park, Scarborough Town Centre, Wexford ...",2.0,Indian Restaurant,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant,Thrift / Vintage Store,Pet Store,Gluten-free Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
11,"Maryvale, Wexford",2.0,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Bakery,Sandwich Place,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


### Cluster 4

In [43]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,
                   toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]

most_common(cluster4)

Most Common Venues in this cluster are:

[Home Service]                   3
[Dumpling Restaurant]            2
[Doner Restaurant]               2
[Dim Sum Restaurant]             2
[Discount Store]                 2
[Eastern European Restaurant]    2
[Donut Shop]                     2
[Drugstore]                      2
[Yoga Studio]                    2
[Dog Run]                        2
dtype: int64


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Newtonbrook, Willowdale",3.0,Home Service,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant
63,Roselawn,3.0,Garden,Home Service,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
91,"Humber Bay, King's Mill Park, Kingsway Park So...",3.0,Home Service,Baseball Field,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio


### Cluster 4
This cluster only contains 1 Neighborhood

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Cloverdale, Islington, Martin Grove, Princess ...",4.0,Filipino Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Fast Food Restaurant
